### Soil Moisture in Portales, New Mexico

**Station Name:**  p038

**Location:**  Portales, New Mexico, USA

**Archive:** [UNAVCO](http://www.unavco.org)

**Ellipsoidal Coordinates:**

- Latitude: 34.14726 degrees

- Longitude: -103.40734 degrees

- Height: 1212.982 meters

[Station Page at UNAVCO](https://www.unavco.org/instrumentation/networks/status/nota/overview/P038)

[Google Maps Link](https://www.google.com/maps/place/34%C2%B008'50.1%22N+103%C2%B024'26.4%22W/@34.14725,-103.4073333,17z/data=!3m1!4b1!4m5!3m4!1s0x0:0x64e449f205085274!8m2!3d34.14725!4d-103.4073333) 

<p align="center">
<img src="../../../data/p038.png" width="500"/>
</P>

P038 was a PBO site. The data from 2017 will be analyzed here as a test case. We will start by analyzing the data 
using the normal reflector height (GNSS-IR) processing. Then we will use those results to run the soil moisture code.

First, start by importing everything we need to run GNSS-IR:

In [ ]:
import json
import os
import sys

import seaborn as sns;

sns.set_theme(style="whitegrid");

# We are including our repository bin to the system path so that we can import the following python modules
bin_path = os.path.abspath(os.path.join('../../../bin'))
if bin_path not in sys.path:
    sys.path.append(bin_path)

import gnssrefl_helpers

#Making sure environment variables are set - this is required to run the gnssrefl code
exists = gnssrefl_helpers.check_environment()
if exists == False:
    gnssrefl_helpers.set_environment(refl_code="../../..", orbits="../../../orbits", exe="../../../bin/exe")

# Set local variable of refl_code location
refl_code_loc = os.environ['REFL_CODE']

# import gnssrefl functions
from gnssrefl.rinex2snr_cl import rinex2snr
from gnssrefl.make_json_input import make_json
from gnssrefl.gnssir_cl import gnssir
from gnssrefl.vwc_input import vwc_input
from gnssrefl.quickPhase import quickphase
from gnssrefl.vwc import vwc
from gnssrefl.installexe_cl import installexe

#@formatter:off
%matplotlib inline

In [ ]:
# import the crx2rnx file which is dependant on your working OS - this is required to run the gnssrefl code
# If in docker environment, then you do not need to download crxnrnx
try:
    os.environ['DOCKER']
except KeyError:
    sys = gnssrefl_helpers.get_sys()
    installexe(sys)

#### Step 1: GNSS-IR
Begin by generating the SNR files. Although typically PBO sites do not have L2C 
data in their low-rate RINEX files, UNAVCO is providing these data in the "special" archive section
so that people can test out this code.

In [ ]:
# Set station, year, doy, and doy_end as variables so they can be used throughout the rest of this example
station = 'p038'
year=2017
doy=1
doy_end=365

rinex2snr(station=station, year=year, doy=doy, doy_end=doy_end, archive='special')

If you want to remind yourself why the L2C data are superior to the L1 data recorded for this receiver, use <code>quickLook</code>. You will need to import it from gnssrefl.

The analysis parameters are set up with <code>make_json</code>. While ordinarily you need to input 
the station latitude, longitude, and ellipsoidal height for this code, if the station is in the <code>gnssrefl</code> database (this station is), you can 
put zero values there instead. We only need the L2C data, so have set the parameter accordingly.

In [ ]:
make_json(station, 34.14726, -103.40734, 1212.977, l2c=True)

# This is the json file that was created with the defaults/parameters you set above
json_file = f'{refl_code_loc}/input/{station}.json'
with open(json_file, "r") as myfile:
    file = json.load(myfile)
    
file

Now we run <code>gnssir</code>. This will be needed for estimate a priori reflector heights for the soil moisture code.

In [ ]:
gnssir(station, year, doy, doy_end=doy_end)

#### Step 2: Soil Moisture

[Please read the soil moisture user manual.](https://github.com/kristinemlarson/gnssrefl/blob/master/docs/README_vwc.md) It is very short and has a lot of tips that will save you time.

We need a list of satellite tracks to use:

In [ ]:
# remember you can see what parameters are accepted by running: vwc_input?

vwc_input(station, 2017, min_tracks=30)

This creates a file that is stored in $REFL_CODE/input/p038_phaseRH.txt

Now we estimate the phase for each satellite track on each day:


In [ ]:
quickphase(station, year, doy, doy_end=doy_end)

Finally, convert the phase to volumetric water content.

This will result in 4 plots:
1. Phase results in geographic coordinates
2. Daily average phase
3. Model inputs
4. Final Results

These will all be saved in $REFL_CODE/Files

In [ ]:
vwc(station, year)

Thank you to Naoya Kadota for test driving this use case.